In [1]:
import pandas as pd
import re
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import io
import matplotlib.pyplot as plt

loading dataset

In [2]:
df = pd.read_excel("/content/spam1.xlsx")

In [3]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


cleaning data removing unwanted features

In [4]:
df = df[['v1', 'v2']]

In [5]:
df.columns = ['Label','Message']

In [6]:
df.head()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
df.dtypes

,0
Label,object
Message,object


In [8]:
df.isnull()

,Label,Message
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
5567,False,False
5568,False,False
5569,False,False
5570,False,False


In [9]:
df.dropna()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [10]:
df['Message'][1]

'Ok lar... Joking wif u oni...'

In [11]:
df.shape[0]

5572

In [12]:
def clean_text_regex(text):
    text = str(text)
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation using regular expression
    return text

In [13]:
df['Message'] = df['Message'].apply(clean_text_regex)

In [14]:
df['Message'][7]

'as per your request melle melle oru minnaminunginte nurungu vettam has been set as your callertune for all callers press 9 to copy your friends callertune'

text preprocessing

In [15]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

tokenizer = Tokenizer(num_words = 20000, oov_token='<OOV>')

In [17]:
train_text = df['Message'].tolist()  # Assuming your DataFrame is named 'df'
tokenizer.fit_on_texts(train_text)

In [18]:
def preprocess_text_tf(text):
    # Tokenize the text using the Keras Tokenizer
    tokenized_text = tokenizer.texts_to_sequences([text])[0]  # Convert text to a sequence of tokens (integers)

    # Convert tokens back to words (vocabulary)
    words = [tokenizer.index_word[i] for i in tokenized_text]

    # Remove stopwords using NLTK
    words = [word for word in words if word not in stop_words]

    # Lemmatize the words using NLTK
    words_lemmatized = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words_lemmatized)

In [19]:
processed_texts = [preprocess_text_tf(text) for text in train_text]

In [20]:
processed_texts

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry questionstd txt ratetcs apply 08452810075over18s',
 'u dun say early hor u c already say',
 'nah dont think go usf life around though',
 'freemsg hey darling 3 week word back id like fun still tb ok xxx std chgs send å150 rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press 9 copy friend callertune',
 'winner valued network customer selected receivea å900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobile 11 month u r entitled update latest colour mobile camera free call mobile update co free 08002986030',
 'im gonna home soon dont want talk stuff anymore tonight k ive cried enough today',
 'six chance win cash 100 20000 pound txt csh11 send 87575 cost 150pday 6days 16

In [21]:
df['processed_message'] = processed_texts

training trditional ml model

In [22]:
X_train, X_test, y_train, y_test = train_test_split(df['processed_message'], df['Label'], test_size=0.2)

In [23]:
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [24]:
model_ml = MultinomialNB()
model_ml.fit(X_train_vectorized, y_train)

MultinomialNB()

In [26]:
y_pred = model_ml.predict(X_test_vectorized)
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99       971
        spam       0.96      0.90      0.93       144

    accuracy                           0.98      1115
   macro avg       0.97      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115

Accuracy: 0.9820627802690582


Training deep learning model

In [27]:
word_index = tokenizer.word_index
word_index

{'<OOV>': 1,
 'to': 2,
 'i': 3,
 'you': 4,
 'a': 5,
 'the': 6,
 'u': 7,
 'and': 8,
 'is': 9,
 'in': 10,
 'me': 11,
 'my': 12,
 'for': 13,
 'your': 14,
 'of': 15,
 'it': 16,
 'call': 17,
 'have': 18,
 'on': 19,
 'that': 20,
 'are': 21,
 'now': 22,
 '2': 23,
 'im': 24,
 'so': 25,
 'not': 26,
 'but': 27,
 'or': 28,
 'can': 29,
 'at': 30,
 'do': 31,
 'get': 32,
 'be': 33,
 'ur': 34,
 'with': 35,
 'if': 36,
 'will': 37,
 'just': 38,
 'we': 39,
 'no': 40,
 'this': 41,
 'its': 42,
 'up': 43,
 '4': 44,
 'when': 45,
 'dont': 46,
 'go': 47,
 'from': 48,
 'ok': 49,
 'ltgt': 50,
 'free': 51,
 'all': 52,
 'out': 53,
 'how': 54,
 'what': 55,
 'know': 56,
 'like': 57,
 'got': 58,
 'ill': 59,
 'good': 60,
 'then': 61,
 'was': 62,
 'come': 63,
 'am': 64,
 'only': 65,
 'time': 66,
 'day': 67,
 'there': 68,
 'love': 69,
 'want': 70,
 'send': 71,
 'text': 72,
 'he': 73,
 'as': 74,
 'going': 75,
 'by': 76,
 'one': 77,
 'about': 78,
 'need': 79,
 'txt': 80,
 'home': 81,
 'lor': 82,
 'see': 83,
 'sorry': 84,

In [28]:
train_seq = tokenizer.texts_to_sequences(train_text)
train_pad = pad_sequences(train_seq)

In [29]:
df['Label'] = df['Label'].map({'ham': 0, 'spam': 1})

In [30]:
train_label = [df['Label'][x] for x in range(0,df.shape[0])]

In [31]:
train_label = np.array(train_label)

In [32]:
train_pads, test_pad, train_labels, test_labels = train_test_split(train_pad, train_label, test_size=0.2, random_state=108)

In [33]:
train_padded, val_pad, train_lab, val_labels = train_test_split(train_pads, train_labels, test_size=0.2, random_state=108)

In [34]:
vocab_size = 10000
embedding_dim = 16
max_len = 132

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_len),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(train_padded, train_lab, epochs=10, validation_data=(val_pad, val_labels))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8288 - loss: 0.4493 - val_accuracy: 0.8879 - val_loss: 0.3465
Epoch 2/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8644 - loss: 0.3848 - val_accuracy: 0.8879 - val_loss: 0.3405
Epoch 3/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8689 - loss: 0.3730 - val_accuracy: 0.8879 - val_loss: 0.3385
Epoch 4/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8709 - loss: 0.3650 - val_accuracy: 0.8879 - val_loss: 0.3354
Epoch 5/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8768 - loss: 0.3536 - val_accuracy: 0.8879 - val_loss: 0.3274
Epoch 6/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8691 - loss: 0.3612 - val_accuracy: 0.8879 - val_loss: 0.3287
Epoch 7/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8720 - loss: 0.3497 - val_accuracy: 0.8879 - val_loss: 0.3094
Epoch 8/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8610 - loss: 0.3489 - val_accuracy: 

In [38]:
print('Deep neural networl:')
model.evaluate(test_pad,test_labels)

Deep neural networl:
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9189 - loss: 0.2202 


[0.23012131452560425, 0.907623291015625]

In [36]:
vocab_size = 20000
embedding_dim = 64
max_len = 132

model1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_len),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model1.fit(train_padded, train_lab, epochs=10, validation_data=(val_pad, val_labels))

Epoch 1/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 15s 115ms/step - accuracy: 0.8760 - loss: 0.3630 - val_accuracy: 0.9798 - val_loss: 0.0749
Epoch 2/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 25s 157ms/step - accuracy: 0.9824 - loss: 0.0628 - val_accuracy: 0.9854 - val_loss: 0.0547
Epoch 3/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 15s 111ms/step - accuracy: 0.9960 - loss: 0.0207 - val_accuracy: 0.9854 - val_loss: 0.0492
Epoch 4/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 20s 102ms/step - accuracy: 0.9987 - loss: 0.0065 - val_accuracy: 0.9865 - val_loss: 0.0534
Epoch 5/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 12s 104ms/step - accuracy: 0.9962 - loss: 0.0089 - val_accuracy: 0.9854 - val_loss: 0.0568
Epoch 6/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 21s 111ms/step - accuracy: 0.9996 - loss: 0.0020 - val_accuracy: 0.9843 - val_loss: 0.0772
Epoch 7/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 20s 107ms/step - accuracy: 0.9998 - loss: 6.7709e-04 - val_accuracy: 0.9843 - val_loss: 0.0772
Epoch 8/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 21s 113ms/step - accuracy: 1.0000 - los

In [39]:
print("LSTM model:")
model1.evaluate(test_pad,test_labels)

LSTM model:
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9820 - loss: 0.0935


[0.08513805270195007, 0.9829596281051636]